# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

## Esquema relacional:

Person(__id__, gender, age)  
Medication(__code__, name)  
Medication-use(*person_id*, *drug_code*, days_use)  

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [20]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Medicamento;
DROP TABLE IF EXISTS UsoMedicamento;

In [21]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

In [22]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [23]:
CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  Nome VARCHAR(80) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

In [24]:
CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodigoMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodigoMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [25]:
SELECT * FROM Medicamento;

In [26]:
SELECT * FROM UsoMedicamento;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [27]:
SELECT * FROM Pessoa 
    WHERE Idade > 60;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [28]:
SELECT Idade, COUNT(*) Quantidade FROM Pessoa 
    GROUP BY Idade;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [29]:
SELECT P.Id, P.Idade, U.CodigoMedicamento, U.DiasUso 
    FROM Pessoa AS P, UsoMedicamento AS U 
        WHERE P.Id = U.IdPessoa;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [30]:
SELECT P.Id, P.Idade, M.Nome, U.DiasUso 
    FROM Pessoa P, UsoMedicamento U, Medicamento M 
        WHERE P.Id = U.IdPessoa 
            AND U.CodigoMedicamento = M.Codigo;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [31]:
SELECT M.Nome, COUNT(*) Quantidade 
    FROM UsoMedicamento U, Medicamento M 
        WHERE U.CodigoMedicamento = M.Codigo 
            AND U.DiasUso > 0 
    GROUP BY M.Nome;

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [32]:
SELECT M.Nome, COUNT(*) Quantidade 
    FROM UsoMedicamento U, Medicamento M 
        WHERE U.CodigoMedicamento = M.Codigo 
            AND U.DiasUso > 2000 
    GROUP BY M.Nome;